In [1]:
import atoti as tt

session = tt.create_session()

Welcome to atoti 0.5.1!

By using this community edition, you agree with the license available at https://www.atoti.io/eula.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


In [2]:
countries_store = session.read_csv("data/countries.csv", keys=["Three_Letter_Country_Code"])

In [3]:
countries_store.head()

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Country_Number
Three_Letter_Country_Code,,,,,
AFG,Asia,AS,"Afghanistan, Islamic Republic of",AF,4
ALB,Europe,EU,"Albania, Republic of",AL,8
ATA,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,10
DZA,Africa,AF,"Algeria, People's Democratic Republic of",DZ,12
ASM,Oceania,OC,American Samoa,AS,16


In [4]:
flags_store = session.read_csv("data/flags.csv", keys=["Flag Code"])

In [5]:
flags_store.head()

,Flag Name
Flag Code,
E,Estimated value
B,Break
I,Incomplete data
n,National estimate


In [6]:
variables_store = session.read_csv("data/variables.csv", keys=["Variable Code"])

In [7]:
variables_store.head()

,Variable Name,Unit Code
Variable Code,,
CO2_PBPROD,"Production-based CO2 productivity, GDP per uni...",USD_KG
CO2_PBEMCAP,"Production-based CO2 intensity, energy-related...",TONNE
NRG_PROD,"Energy productivity, GDP per unit of TPES",USD
NRG_INT,"Energy intensity, TPES per capita",TOE
RE_TPES,"Renewable energy supply, % total energy supply",PC


In [8]:
main_store = session.read_csv("data/green-growth.csv", keys=["Country Code", "Year"])

In [9]:
main_store.head()

,,Flag Codes,GDP per unit of energy-related CO2 emissions (USD per kg),Energy-related CO2 per capita (Tonne),GDP per unit of TPES (USD),TPES per capita (Tonne of oil equivalent),Renewable energy supply (% of total energy supply),Renewable electricity (% of total electricity generation),Total freshwater abstraction (m3 per capita),Water stress (total freshwater abstraction as % total available renewable resources),Water stress (total freshwater abstraction as % total internal renewable resources),...,Emissions priced > 30€ per tonne of CO2 (% of total emissions),"Farmland Birds (Index, 2000=100)",Marine protected area (% of total exclusive economic zone),Intact forest landscape loss (% since 2000),"Forest Birds (Index, 2000=100)",Electricity support (% of total fossil fuel support),CO2 emissions per unit of GDP,Renewable energy supply (excluding solid biofuels) (% of total energy supply),Naturally regenerating forests (% of total forest area),Forests with long-term management plans (% of total forest area)
Country Code,Year,,,,,,,,,,,,,,,,,,,,,
BDI,2015,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.596016,NaN
BWA,2019,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MNG,2019,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDV,2019,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SUR,2019,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
main_store.join(countries_store, mapping={"Country Code": "Three_Letter_Country_Code"})

In [11]:
cube = session.create_cube(main_store)

In [12]:
h = cube.hierarchies
lvl = cube.levels
m = cube.measures

In [13]:
h

Hierarchies()

In [14]:
h["Country"] = [lvl["Continent_Name"], lvl["Country_Name"]]

In [15]:
del h["Two_Letter_Country_Code"]

In [16]:
del h["Country Code"]

In [17]:
del h["Continent_Name"]

In [18]:
del h["Continent_Code"]

In [19]:
del h["Country_Number"]

In [20]:
del h["Country_Name"]

In [21]:
summableMeasures = [
    "Total primary energy supply (tonne of oil equivalent)",
    "Production-based CO2 emissions (tonne)",
    "Demand-based CO2 emissions (tonne)",
    "Forest resource stocks (m3)",
    "Population",
    "Net migration"
]

In [22]:
for key in m:
    if "%" in key:
        if key.endswith(".SUM"):
            del m[key]
        else:
            m[key].formatter = "DOUBLE[#.##%]"
    else:
        isSummable = any(key.startswith(x) for x in summableMeasures)
        if ((isSummable and key.endswith(".MEAN")) or (not isSummable and key.endswith(".SUM"))):
            del m[key]
        

In [23]:
h["Year"].slicing = True

In [24]:
session.url

'http://localhost:56935'

In [25]:
cube